This workbook requires no additional modules and should work with any _stable_ version of Julia.<br/>
The _ModInts_ module is from the Julia examples directory but is reproduced here in the _code_ subdirectory.<br/><br/>


# Why is Julia fast and can we retrofit to Python? 

## Type Inference

To generate fast code for a function f(x,y), the compiler needs to be able to infer the types of variables in f,
map them to hardware types (registers) where possible, and call specialized code paths for those types.

At compile-time, the compiler generally only knows types of x,y, but not the values, and it needs to be able
to cascade this information to infer types throughout f and in any functions called by f.

Julia and its standard library are designed so type inference is possible for code following straightforward rules.

Sometimes this requires subtle choices that would be painful to retrofit onto an existing language.



### _The return type of a function should ONLY depend on the types of its arguments_

### This needs to be run in Python

```python
# Import NUMPY for square roots

import numpy as np

# Look at type (in)stability

type(2 ** 3)
type(2 ** -3)

# How this is correct in Python but not in MATLAB

np.sqrt(1)
np.sqrt(-1)
np.sqrt(-1 + 0j)

```

### Let's see how Julia deals with this

In [ ]:
typeof(2^3)

In [ ]:
typeof(2^-3)

In [ ]:
sqrt(1)

In [ ]:
sqrt(-1)

In [ ]:
sqrt(-1 + 0im)

### and with the factorial

In [ ]:
fac(n) = (n < 2) ? 1 : n*fac(n-1);
fac(20)

In [ ]:
fac(21)

### but

In [ ]:
fac(big(21))

In [ ]:
factorial(21)   # in the Base.combinatorics module

- Not all mathematical operations on numbers in Julia produce the same _type_ of result.
- But Julia is consistant or an exception is raised.
- The outcome(s) are not without being the subject of a _great_ deal of debate.

In [ ]:
2/4

In [ ]:
4/2

In [ ]:
4 ÷ 2            # x ÷ y    =>    div(x,y)  or  x div y

In [ ]:
4//11 / 2//7     # divide 2 rationals

In [ ]:
typeof(4//11 / 2//7)

In [ ]:
typeof(4//11 ÷ 2//7)

In [ ]:
4//11 ÷ 2//7

In [ ]:
typeof(4.0 ÷ 3.0)

---

### Delegation O-O paradigm, as contrasted with inheritence or association

We will look at how this works using one of the original Julia examples: Modular Integers

In [ ]:
# We need to be able to import the module by munging the LOAD PATH

In [ ]:
cd("/Users/malcolm/notebooks.git/PYD/code");  # This is dependent on YOUR directory structure

In [ ]:
push!(LOAD_PATH,".")

In [ ]:
import ModInts;    # "import" does not add functions to the MAIN class  
ms = ModInts       # so we need to assign an identifier, in a similar fash to Python's "as" statement

In [ ]:
; cat modints.jl

In [ ]:
m1 = ms.ModInt{13}(8);
m2 = ms.ModInt{13}(4);
m3 = ms.ModInt{13}(3);

In [ ]:
m1 + m2*m3

In [ ]:
inv(m3)

In [ ]:
m1 + 1

In [ ]:
m1 + 1//2

In [ ]:
m1 + 2.0

In [ ]:
m1 + int(2.0)

In [ ]:
mm = [ms.ModInt{13}(rand(1:13)) for i = 1:100] 

In [ ]:
ma = reshape(mm,10,10)

In [ ]:
mx = [ms.ModInt{13}(rand(1:13)) for i = 1:10]

In [ ]:
mb = mx' .* ma .* mx 

In [ ]:
mc = map(x -> x^3, mb)

In [ ]:
mc[1:5,1:5]

## Staged code generation

### Parsed code ------> AST ------> AST (typed) ------> LLVM IR ------> [Native code]

```julia
# 1 == 1 parsed as ==(1,1)

# Try some of the following:

@code_lowered 1 == 1       # or  ==(1,1)
@code_typed   1 == 1
@code_llvm    1 == 1  
@code_native  1 == 1

```

In [ ]:
@code_native 1 == 1

In [ ]:
@code_native 1 == 1.0

In [ ]:
# By the way

1 == 1.0

In [ ]:
1 === 1.0

In [ ]:
# Insert a simple insert function

incr(x) = x + 1

In [ ]:
@code_native incr(1)   # Code for an integer argument

In [ ]:
@code_native incr(1.0)  # Code for an float

In [ ]:
incr(m1)    # This works too

In [ ]:
@code_native incr(m1)   # Here is the 'native' code

In [ ]:
# We help the compiler if we restrict (type) the arguments
#
# Define a fibonanicci function (recursively)

function fib(n :: Integer)
  @assert n > 0
  return (n < 3) ? 1 : fib(n-1) + fib(n-2)
end

fib(20)

In [ ]:
fib(-20)

In [ ]:
fib(2.3)

In [ ]:
macroexpand(:(@assert n > 0))

In [ ]:
@code_native(fib(20))

In [ ]:
macroexpand(:(@printf "The value of fib(%d) is %d" 20 fib(20)))

---

### Library calls have zero-overhead

In [ ]:
systime() = ccall((:time,("libc")), Int32, ())

In [ ]:
systime()

In [ ]:
@code_native systime()